In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14


In [2]:
import numpy as np
import pandas as pd
from collections import namedtuple
import os
import glob

In [22]:
crystal = namedtuple("crystal", "wl_nm td_ns N")
pde = namedtuple("pde", "bgo lyso csiTl csi")
RFWHM = namedtuple("RFWHM", "bgo lyso csiTl csi")

In [23]:
bgo = crystal(wl_nm=480, td_ns = 300, N=9e+3)
lyso = crystal(wl_nm=410, td_ns = 40, N=25e+3)
csiTl = crystal(wl_nm=550, td_ns = 900, N=55e+3)
csi = crystal(wl_nm=350, td_ns = 800, N=100e+3)

In [53]:
r1306QE = pde(bgo = 0.18, lyso = 0.25, csiTl = 0.09, csi=0.15)
s14160 = pde(bgo = 0.45, lyso = 0.50, csiTl = 0.35, csi=0.40)

In [25]:
Riee_660 = RFWHM(bgo=0.09, lyso=0.08, csiTl=0.057, csi=0.05)
Rmat_660 = RFWHM(bgo=0.166, lyso=0.13, csiTl=0.071, csi=0.05)

In [68]:
def FF(R, Npe):
    return R**2*Npe/2.355**2

In [76]:
def RF(F, Npe):
    return 2.355 * np.sqrt(F/Npe)

In [26]:
def nphe(yMeV, pDE, xE):
    """
    nphe = yMeV *  pDE *  xE, where:
    yMeV: scintillation yield/MeV
    pDE : sensor PDE
    xE  : energy/(1 MeV). 
    """
    return yMeV *  pDE *  xE

In [29]:
def rfwhm(nphe):
    """
    Resolution due to Poisson statistics of Number of photoelectrons
    """
    return 2.355/np.sqrt(nphe)
    

In [32]:
def rs(R, RM):
    """
    Intrinsic (scintillator) resolution 
    """
    return np.sqrt(R**2 - RM**2)

In [36]:
def xrs(yMeV, pDE, xE, R):
    npe = nphe(yMeV, pDE, xE) 
    Rm  = rfwhm(npe)
    return npe, Rm, rs(R, Rm)
    

In [45]:
def xr(yMeV, pDE, xE, Rs):
    npe = nphe(yMeV, pDE, xE) 
    Rm  = rfwhm(npe)
    return npe, Rm, np.sqrt(Rm**2 + Rs**2)

## Using Article in IEEE Transactions on Nuclear Science · January 1996DOI: 10.1109/23.489415 · Source: IEEE Xplore

### CsI(Tl)

In [46]:
nphe_660_r1306QE_csiTl, Rm_660_r1306QE_csiTl, Rs_660_r1306QE_csiTl = xrs(csiTl.N, r1306QE.csiTl, 0.66, Riee_660.csiTl)

In [47]:
print(f" Npe CsI(Tl) at 660 keV PMT: r1306QE = {nphe_660_r1306QE_csiTl}")
print(f" RM CsI(Tl) at 660 keV PMT: r1306QE = {Rm_660_r1306QE_csiTl}")
print(f" Rs CsI(Tl) at 660 keV PMT: r1306QE = {Rs_660_r1306QE_csiTl}")

 Npe CsI(Tl) at 660 keV PMT: r1306QE = 3267.0
 RM CsI(Tl) at 660 keV PMT: r1306QE = 0.041201814664896017
 Rs CsI(Tl) at 660 keV PMT: r1306QE = 0.039387948262375384


In [48]:
nphe_550_s14160_csiTl, Rm_550_s14160_csiTl, R_550_s14160_csiTl = xr(csiTl.N, s14160.csiTl, 0.511, Rs_660_r1306QE_csiTl)

In [49]:
print(f" Npe CsI(Tl) at 511 keV SiPM: s14160 = {nphe_550_s14160_csiTl}")
print(f" RM CsI(Tl) at 511 keV SiPM: s14160 = {Rm_550_s14160_csiTl}")
print(f" R CsI(Tl) at 511 keV SiPM: s14160 = {R_550_s14160_csiTl}")

 Npe CsI(Tl) at 511 keV SiPM: s14160 = 9836.75
 RM CsI(Tl) at 511 keV SiPM: s14160 = 0.023744612935451125
 R CsI(Tl) at 511 keV SiPM: s14160 = 0.0459914895581123


In [72]:
F_csiTl = FF(R_550_s14160_csiTl, nphe_550_s14160_csiTl)
print(f" F CsI(Tl) at 511 keV SiPM: s14160 = {F_csiTl}")

 F CsI(Tl) at 511 keV SiPM: s14160 = 3.7516711382012216


In [78]:
print(f" Resolution (from F) CsI(Tl) at 511 keV SiPM: s14160 = {RF(F_csiTl, nphe_550_s14160_csiTl):.3f}") 

 Resolution (from F) CsI(Tl) at 511 keV SiPM: s14160 = 0.046


### BGO

In [54]:
nphe_660_r1306QE_bgo, Rm_660_r1306QE_bgo, Rs_660_r1306QE_bgo = xrs(bgo.N, r1306QE.bgo, 0.66, Riee_660.bgo)
print(f" Npe BGO at 660 keV PMT: r1306QE = {nphe_660_r1306QE_bgo}")
print(f" RM BGO at 660 keV PMT: r1306QE = {Rm_660_r1306QE_bgo}")
print(f" Rs BGO at 660 keV PMT: r1306QE = {Rs_660_r1306QE_bgo}")

 Npe BGO at 660 keV PMT: r1306QE = 1069.2
 RM BGO at 660 keV PMT: r1306QE = 0.07202137963534942
 Rs BGO at 660 keV PMT: r1306QE = 0.05397148205692405


In [58]:
nphe_550_s14160_bgo, Rm_550_s14160_bgo, R_550_s14160_bgo = xr(bgo.N, s14160.bgo, 0.511, Rs_660_r1306QE_bgo)
print(f" Npe BGO at 511 keV SiPM: s14160 = {nphe_550_s14160_bgo}")
print(f" RM BGO at 511 keV SiPM: s14160 = {Rm_550_s14160_bgo}")
print(f" R BGO at 511 keV SiPM: s14160 = {R_550_s14160_bgo}")

 Npe BGO at 511 keV SiPM: s14160 = 2069.55
 RM BGO at 511 keV SiPM: s14160 = 0.05176699431461755
 R BGO at 511 keV SiPM: s14160 = 0.07478464130949965


In [73]:
F_bgo = FF(R_550_s14160_bgo, nphe_550_s14160_bgo)
print(f" F BGO at 511 keV SiPM: s14160 = {F_bgo}")

 F BGO at 511 keV SiPM: s14160 = 2.0869830910836633


In [79]:
print(f" Resolution (from F) BGO at 511 keV SiPM: s14160 = {RF(F_bgo, nphe_550_s14160_bgo):.3f}") 

 Resolution (from F) BGO at 511 keV SiPM: s14160 = 0.075


### LYSO

In [59]:
nphe_660_r1306QE_lyso, Rm_660_r1306QE_lyso, Rs_660_r1306QE_lyso = xrs(lyso.N, r1306QE.lyso, 0.66, Riee_660.lyso)
print(f" Npe lyso at 660 keV PMT: r1306QE = {nphe_660_r1306QE_lyso}")
print(f" RM lyso at 660 keV PMT: r1306QE = {Rm_660_r1306QE_lyso}")
print(f" Rs lyso at 660 keV PMT: r1306QE = {Rs_660_r1306QE_lyso}")

 Npe lyso at 660 keV PMT: r1306QE = 4125.0
 RM lyso at 660 keV PMT: r1306QE = 0.03666730027000773
 Rs lyso at 660 keV PMT: r1306QE = 0.07110210328048736


In [61]:
nphe_550_s14160_lyso, Rm_550_s14160_lyso, R_550_s14160_lyso = xr(lyso.N, s14160.lyso, 0.511, Rs_660_r1306QE_lyso)
print(f" Npe lyso at 511 keV SiPM: s14160 = {nphe_550_s14160_lyso}")
print(f" RM lyso at 511 keV SiPM: s14160 = {Rm_550_s14160_lyso}")
print(f" R lyso at 511 keV SiPM: s14160 = {R_550_s14160_lyso}")

 Npe lyso at 511 keV SiPM: s14160 = 6387.5
 RM lyso at 511 keV SiPM: s14160 = 0.029466289737932142
 R lyso at 511 keV SiPM: s14160 = 0.07696604005552615


In [74]:
F_lyso = FF(R_550_s14160_lyso, nphe_550_s14160_lyso)
print(f" F LYSO at 511 keV SiPM: s14160 = {F_lyso}")

 F LYSO at 511 keV SiPM: s14160 = 6.822560179260249


In [80]:
print(f" Resolution (from F) LYSO at 511 keV SiPM: s14160 = {RF(F_lyso, nphe_550_s14160_lyso):.3f}") 

 Resolution (from F) LYSO at 511 keV SiPM: s14160 = 0.077


### CsI (Characterization and Optimization of Cryogenic Pure CsI Detector for CLOVERS Experiment)

In [64]:
npe_csi_60keV = 35.2 * 60
R_60keV = 0.069
Rm_csi_60keV = rfwhm(npe_csi_60keV)
Rs_csi_60keV = rs(R_60keV, Rm_csi_60keV)
print(f" Npe CsI at 60 keV = {npe_csi_60keV}")
print(f" RM CsI at 60 keV  = {Rm_csi_60keV}")
print(f" Rs CsI at 60 keV  = {Rs_csi_60keV}")

 Npe CsI at 60 keV = 2112.0
 RM CsI at 60 keV  = 0.05124410997195855
 Rs CsI at 60 keV  = 0.046206505961626426


#### Non prop at 60 keV: Rs ~4.6 % (60 keV). Instead, at 660 (511) keV, Rs = 3.9 %

### CsI calculation:

- Take N = 100E+3 phot/MeV
- Compute nphe using SiPM s14160
- Take Rs = 0.039

In [66]:
nphe_550_s14160_csi = csi.N * s14160.csi * 0.511
print(f" Npe CsI at 511  keV SiPM s14160 = {nphe_550_s14160_csi}")

 Npe CsI at 511  keV SiPM s14160 = 20440.0


In [67]:
Rm_csi_511keV = rfwhm(nphe_550_s14160_csi)
Rs_csi_511keV = Rs_660_r1306QE_csiTl
R_csi_511keV  = np.sqrt(Rm_csi_511keV**2 + Rs_csi_511keV**2)
print(f" Rs CsI at 511 keV = {Rs_csi_511keV}")
print(f" RM CsI at 511 keV  = {Rm_csi_511keV}")
print(f" Rs CsI at 511 keV  = {R_csi_511keV}")

 Rs CsI at 511 keV = 0.039387948262375384
 RM CsI at 511 keV  = 0.016472156724680182
 Rs CsI at 511 keV  = 0.04269358752180456


In [75]:
F_csi = FF(R_csi_511keV, nphe_550_s14160_csi)
print(f" F CsI at 511 keV SiPM: s14160 = {F_csi}")

 F CsI at 511 keV SiPM: s14160 = 6.717758209249291


In [81]:
print(f" Resolution (from F) CsI at 511 keV SiPM: s14160 = {RF(F_csi, nphe_550_s14160_csi):.3f}") 

 Resolution (from F) CsI at 511 keV SiPM: s14160 = 0.043
